# Семинар по рекуррентным нейронным сетям
На этом семинаре мы обучим несколько рекуррентных архитектур для решения задачи сентимент-анализа, то есть предсказания метки тональности предложения.

В общем случае рекуррентная нейронная сеть предназначена для обработки последовательности произвольной длины. Однако при реализации метода оказывается проще зафиксировать длину последовательности (даже в pytorch с их динамическими графами :) Мы так и поступим.

При выполнении задания вы обучите LSTM с разным уровнем "коробочности", а также познакомитесь с различными способами применения дропаута к рекуррентным архитектурам. В рекуррентных архитектурах вариантов, куда можно наложить бинарную маску шума, гораздо больше, чем в нейросетях прямого прохода.

Задание сделано так, чтобы его можно было выполнять на CPU, однако RNN - это ресурсоемкая вещь, поэтому на GPU с ними работать приятнее. Можете попробовать использовать [https://colab.research.google.com](https://colab.research.google.com) - бесплатное облако с GPU.

### Гиперпараметры

In [31]:
vocab_size = 20000
index_from = 3
n_hidden = 32 # 128
n_emb = 32 # 128
seq_len = 32 # 200
# small network on small data for seminar purposes
# after # normal size

batch_size = 128
learning_rate = 0.001
num_epochs = 1 # 30

### Загрузка данных
Функция load_matrix_imdb скачивает матричные данные, перемешивает и загружает их в numpy-массивы.

Если у вас не установлен wget, скачайте [архив imdb.npz](https://s3.amazonaws.com/text-datasets/imdb.npz)

In [32]:
#from rnn_utils import load_matrix_imdb
import numpy as np
import torch
import torch.utils.data
import os
import re
from collections import defaultdict
import operator

In [33]:
def load_matrix_imdb(path='imdb.npz', num_words=None, skip_top=0,
              maxlen=None, seed=113,
              start_char=1, oov_char=2, index_from=3, **kwargs):
    """
    Modified code from Keras
    Loads data matrixes from npz file, crops and pads seqs and returns
    shuffled (x_train, y_train), (x_test, y_test)
    """
    if not os.path.exists(path):
        print("Downloading matrix data into current folder")
        os.system("wget https://s3.amazonaws.com/text-datasets/imdb.npz")
        
    with np.load(path, allow_pickle=True) as f:
        x_train, labels_train = f['x_train'], f['y_train']
        x_test, labels_test = f['x_test'], f['y_test']

    np.random.seed(seed)
    indices = np.arange(len(x_train))
    np.random.shuffle(indices)
    x_train = x_train[indices]
    labels_train = labels_train[indices]

    indices = np.arange(len(x_test))
    np.random.shuffle(indices)
    x_test = x_test[indices]
    labels_test = labels_test[indices]

    xs = np.concatenate([x_train, x_test])
    labels = np.concatenate([labels_train, labels_test])

    if start_char is not None:
        xs = [[start_char] + [w + index_from for w in x] for x in xs]
    elif index_from:
        xs = [[w + index_from for w in x] for x in xs]

    if not num_words:
        num_words = max([max(x) for x in xs])
    if not maxlen:
        maxlen = max([len(x) for x in xs])

    # by convention, use 2 as OOV word
    # reserve 'index_from' (=3 by default) characters:
    # 0 (padding), 1 (start), 2 (OOV)
    xs_new = []
    for x in xs:
        x = x[:maxlen] # crop long sequences
        if oov_char is not None: # replace rare or frequent symbols 
            x = [w if (skip_top <= w < num_words) else oov_char for w in x]
        else: # or filter rare and frequent symbols
            x = [w for w in x if skip_top <= w < num_words]
        x_padded = np.zeros(maxlen)#, dtype = 'int32')
        x_padded[-len(x):] = x
        xs_new.append(x_padded)    
            
    idx = len(x_train)
    x_train, y_train = np.array(xs_new[:idx]), np.array(labels[:idx])
    x_test, y_test = np.array(xs_new[idx:]), np.array(labels[idx:])

    return (x_train, y_train), (x_test, y_test)

In [34]:
np.random.seed(0)
(X_train, y_train), (X_test, y_test) = load_matrix_imdb(num_words=vocab_size,
                                                        maxlen=seq_len)

In [35]:
set(y_train) # binary classification

{0, 1}

In [36]:
X_train.shape, X_test.shape

((25000, 32), (25000, 32))

In [37]:
X_train[0].astype(np.int32) # sequence of coded words

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5],
      dtype=int32)

In [38]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [39]:
train_dset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.long), 
                               torch.tensor(y_train, dtype=torch.long))
train_dset[0]

(tensor([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458, 4468,
           66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,  838,  112,
           50,  670,    2,    9,   35,  480,  284,    5]),
 tensor(1))

In [40]:
test_dset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long), 
                               torch.tensor(y_test, dtype=torch.long))
test_dset[0]

(tensor([    1,   591,   202,    14,    31,     6,   717,    10,    10, 18142,
         10698,     5,     4,   360,     7,     4,   177,  5760,   394,   354,
             4,   123,     9,  1035,  1035,  1035,    10,    10,    13,    92,
           124,    89]),
 tensor(0))

In [41]:
train_loader = torch.utils.data.DataLoader(train_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4,
                         )

In [42]:
test_loader = torch.utils.data.DataLoader(test_dset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=4,
                         )

### Сборка и обучение RNN в pytorch

In [43]:
import os
import torch.optim as optim
import torch.nn as nn

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Наша нейросеть будет обрабатывать входную последовательность по словам (word level). Мы будем использовать простую и стандарную рекуррентную архитектуру для сентимент-анализа: слой представлений, слой LSTM и полносвязный слой, предсказывающий выход по последнему скрытому состоянию.

Ниже дан код для сборки и обучения нашей нейросети. Обратите внимание на ### pay attention here, указывающие на особенности кода при использовании рекуррентных слоев. 

Чтобы потом было удобно проводить сравнение времени работы разных моделей, запустите код ниже и измерьте время обучения на вашей системе.

In [47]:
class RNNClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, \
                 batch_size, rec_layer=nn.LSTM, embedding=nn.Embedding, \
                 dropout=None):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = embedding(vocab_size, embedding_dim)
        if dropout:
            self.rnn = rec_layer(embedding_dim, hidden_dim, dropout=dropout)
        else:
            self.rnn = rec_layer(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
    
    def forward(self, sentences):
        """
        param: sentences: tensor of shape (seq_len, batch_size)
        """
        embedding = self.word_embeddings(sentences) # (seq_len, batch_size)
        out, hidden = self.rnn(embedding) # pay attention here!
        res = self.hidden2label(out[-1])
        return torch.sigmoid(res)
    

[Исходный код LSTM](http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM)

In [48]:
rnn = nn.LSTM(input_size=10, hidden_size=20, num_layers=2)
inp = torch.randn(5, 3, 10)  # (seq_len, bs, inp)
h0 = torch.randn(2, 3, 20)  # (n_layers, batch_size, n_hid)
c0 = torch.randn(2, 3, 20)  # (n_layers, batch_size, n_hid)
output, (hn, cn) = rnn(inp, (h0, c0))

output.shape  # (seq_len, bs, hid)

torch.Size([5, 3, 20])

In [49]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=nn.LSTM,
                       dropout=None).to(device)

In [50]:
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')  # mean => sum / (seq_len * bs)

In [51]:
def train_epoch(train_loader, model, lossfun, optimizer, device):
    model.train()
    for it, traindata in enumerate(train_loader):
        train_inputs, train_labels = traindata
        train_inputs = train_inputs.to(device) 
        train_labels = train_labels.to(device)
        train_labels = torch.squeeze(train_labels)

        model.zero_grad()        
        output = model(train_inputs.t()) # pay attention here!

        loss = lossfun(output.view(-1), train_labels.float())
        loss.backward()
        optimizer.step()

def evaluate(loader, model, lossfun, device):
    model.eval()
    total_acc = 0.0
    total_loss = 0.0
    total = 0.0
    for it, data in enumerate(loader):
        inputs, labels = data
        inputs = inputs.to(device) 
        labels = labels.to(device)
        labels = torch.squeeze(labels)

        output = model(inputs.t()) # pay attention here!
        loss = lossfun(output.view(-1), labels.float())# + torch.norm(WW^T - I)
        total_loss += loss.item()

        # calc testing acc        
        pred = output.view(-1) > 0.5
        correct = pred == labels.byte()
        total_acc += torch.sum(correct).item() / len(correct)

    total = it + 1
    return total_loss / total, total_acc / total
    

def train(train_loader, test_loader, model, lossfun, optimizer, \
          device, num_epochs):
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []
    for epoch in range(num_epochs):
        train_epoch(train_loader, model, lossfun, optimizer, device)
        train_loss, train_acc = evaluate(train_loader, model, lossfun, device)
        train_loss_.append(train_loss)
        train_acc_.append(train_acc)
        test_loss, test_acc = evaluate(test_loader, model, lossfun, device)
        test_loss_.append(test_loss)
        test_acc_.append(test_acc)

        print(f'Epoch: {epoch+1:3d}/{num_epochs:3d} '
              f'Training Loss: {train_loss_[epoch]:.3f}, Testing Loss: {test_loss_[epoch]:.3f}, '
              f'Training Acc: {train_acc_[epoch]:.3f}, Testing Acc: {test_acc_[epoch]:.3f}')

    return train_loss_, train_acc_, test_loss_, test_acc_

In [52]:
%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/  1 Training Loss: 83.627, Testing Loss: 85.025, Training Acc: 0.621, Testing Acc: 0.600
CPU times: user 3.29 s, sys: 939 ms, total: 4.23 s
Wall time: 7.84 s


Нерегуляризованные LSTM часто быстро переобучаются (и мы это видим по точности на контроле). Чтобы с этим бороться, часто используют L2-регуляризацию и дропаут.
Однако способов накладывать дропаут на рекуррентный слой достаточно много, и далеко не все хорошо работают. По [ссылке](https://medium.com/@bingobee01/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b) доступен хороший обзор дропаутов для RNN.

Мы реализуем два варианта дропаута для RNN (и третий дополнительно). Заодно увидим, что для реализации различных усовершенствований рекуррентной архитектуры приходится "вскрывать" слой до различной "глубины".

### Реализация дропаута по статье Гала и Гарамани (Variational Dropout)
Начнем с дропаута, описанного в [статье Гала и Гарамани](https://arxiv.org/abs/1512.05287).
Для этого нам потребуется перейти от использования слоя nn.LSTM, полностью скрывающего от нас рекуррентную логику, к использованию слоя nn.LSTMCell, обрабатывающего лишь один временной шаг нашей последовательности (а всю логику вокруг придется реализовать самостоятельно). 

Допишите класс RNNLayer. При dropout=0 ваш класс должен работать как обычный слой LSTM, а при dropout > 0 накладывать бинарную маску на входной и скрытый вектор на каждом временном шаге, причем эта маска должна быть одинаковой во все моменты времени.

Дропаут Гала и Гарамани в виде формул (m обознаает маску дропаута):
$$
h_{t-1} = h_{t-1}*m_h, \, x_t = x_t * m_x
$$
(далее обычный шаг рекуррентной архитектуры, например LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$

In [53]:
def init_h0_c0(num_objects, hidden_size, some_existing_tensor):
    """
    return h0 and c0, use some_existing_tensor.new_zeros() to gen them
    h0 shape: num_objects x hidden_size
    c0 shape: num_objects x hidden_size
    """
    ### your code here
    h0 = some_existing_tensor.new_zeros((num_objects, hidden_size))
    c0 = some_existing_tensor.new_zeros((num_objects, hidden_size))
    return h0, c0

In [54]:
def gen_dropout_mask(input_size, hidden_size, is_training, p, some_existing_tensor):
    """
    is_training: if True, gen masks from Bernoulli
                 if False, gen masks consisting of (1-p)
    
    return dropout masks of size input_size, hidden_size if p is not None
    return one masks if p is None
    """
    ### your code here
    mask0 = some_existing_tensor.new_ones(input_size)
    mask1 = some_existing_tensor.new_ones(hidden_size)
    if is_training and p is not None and p > 0:
        keep_prob = 1 - p
        mask0 = torch.bernoulli(mask0 * keep_prob) / keep_prob
        mask1 = torch.bernoulli(mask1 * keep_prob) / keep_prob
    return mask0, mask1

In [29]:
class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=None):
        super(RNNLayer, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.dropout = dropout
        self.rnn_cell = nn.LSTMCell(input_size, hidden_size)
        
    def forward(self, inp):
        # initialize h_0, c_0
        h_0, c_0 = init_h0_c0(inp.shape[1], self.hidden_size, inp)
        
        # gen masks
        input_mask, hidden_mask = gen_dropout_mask(self.input_size, \
                                                   self.hidden_size, \
                                                   self.training, \
                                                   self.dropout, \
                                                   inp)
        
        
        ### your code here
        ### implement recurrent logic and return what nn.LSTM returns
        ### do not forget to apply generated dropout masks!
        out = []
        h, c = h_0, c_0
        for i in range(inp.shape[0]):
            h, c = self.rnn_cell(inp[i] * input_mask, (h * hidden_mask, c))
            out.append(h)
        return out, (h, c)

Протестируйте реализованную модель с выключенным дропаутом (слой RNNLayer надо передать в RNNClassifier в качестве rec_layer). Замерьте время обучения (%time). Сильно ли оно увеличилось по сравнению с nn.LSTM (LSTM "из коробки")?

In [55]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=RNNLayer,
                       dropout=None).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/  1 Training Loss: 82.116, Testing Loss: 83.893, Training Acc: 0.635, Testing Acc: 0.612
CPU times: user 4.09 s, sys: 1.11 s, total: 5.2 s
Wall time: 8.61 s


Протестируйте полученную модель c dropout=0.5, вновь замерив время обучения. Получилось ли побороть переобучение? Сильно ли дольше обучается данная модель по сравнению с предыдущей? (доп. время тратится на генерацию масок дропаута).

In [57]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=RNNLayer,
                       dropout=0.3).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/  1 Training Loss: 87.296, Testing Loss: 87.694, Training Acc: 0.561, Testing Acc: 0.544
CPU times: user 4.1 s, sys: 1.1 s, total: 5.2 s
Wall time: 8.69 s


In [59]:
model

RNNClassifier(
  (word_embeddings): Embedding(20000, 32)
  (rnn): RNNLayer(
    (rnn_cell): LSTMCell(32, 32)
  )
  (hidden2label): Linear(in_features=32, out_features=1, bias=True)
)

### Реализация дропаута по статье Гала и Гарамани. Дубль 2

<начало взлома pytorch>

При разворачивании цикла по времени средствами python обучение рекуррентной нейросети сильно замедляется. Однако для реализации дропаута Гала и Гарамани необязательно явно задавать в коде домножение нейронов на маски. Можно схитрить и обойтись использованием слоя nn.LSTM: перед вызовом forward слоя nn.LSTM подменять его веса на веса, домноженные по строкам на маски. А обучаемые веса хранить отдельно. Именно так этот дропаут реализован в библиотеке fastai, код из которой использован в ячейке ниже.

Такой слой реализуется в виде обертки над nn.LSTM. Допишите класс:

In [27]:
import warnings

In [28]:
class FastRNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0):
        super(FastRNNLayer, self).__init__()
        self.model = nn.LSTM(input_size, hidden_size)
        self.dropout = dropout
        self.layer_names = ['weight_hh_l0', 'weight_ih_l0']
        for layer in self.layer_names:
            # Makes a copy of the weights of the selected layers.
            w = getattr(self.model, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            
    def _setweights(self):
        "Apply dropout to the raw weights."
        ### your code here
        ### generate input_mask and hidden_mask (use function gen_dropout_mask)
        input_mask, hidden_mask = gen_dropout_mask(self.model.input_size, \
                                                   self.model.hidden_size, \
                                                   self.training, \
                                                   self.dropout, \
                                                   self.model.weight_hh_l0.data)
        for layer, mask in zip(self.layer_names, (hidden_mask, input_mask)):
            raw_w = getattr(self, f'{layer}_raw')
            self.model._parameters[layer] = raw_w * mask

    def forward(self, *args):
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.filterwarnings("ignore")
            
            ### your code here
            ### set new weights of self.module and call its forward
            self._setweights()
            # lstm.param.data *= mask
            return self.model.forward(*args)

    def reset(self):
        if hasattr(self.model, 'reset'): self.model.reset()

Протестируйте полученный слой (вновь подставив его в RNNClassifier в качестве rec_layer) с dropout=0.5. Сравните время обучения с предыдущими моделями. Проследите, чтобы качество получилось такое же, как при первой реализации этого дропаута.

In [29]:
warnings.filterwarnings('ignore')


model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=FastRNNLayer,
                       dropout=0.5).to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')


%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs + 1)

Epoch:   1/  2 Training Loss: 86.312, Testing Loss: 87.327, Training Acc: 0.590, Testing Acc: 0.560
Epoch:   2/  2 Training Loss: 79.340, Testing Loss: 83.651, Training Acc: 0.667, Testing Acc: 0.613
CPU times: user 38.4 s, sys: 57.6 s, total: 1min 35s
Wall time: 14.5 s


</конец взлома pytorch>

### Реализация дропаута по статье Семениуты и др
Перейдем к реализации дропаута для LSTM по статье [Semeniuta et al](http://www.aclweb.org/anthology/C16-1165). 

Этот метод применения дропаута не менее популярен, чем предыдущий. Его особенность состоит в том, что он придуман специально для гейтовых архитектур. В контексте LSTM этот дропаут накладывается только на информационный поток (m_h - маска дропаута):
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot g \odot {\bf m_h} \quad
h_t =  o \odot tanh(c_t) \nonumber
$$
На входы $x_t$ маска накладывается как в предыдущем дропауте. Впрочем, на входы маску можно наложить вообще до вызова рекуррентного слоя.

Согласно статье, маска дропаута может быть как одинаковая, так и разная для всех моментов времени. Мы сделаем одинаковую.

Для реализации этого дропаута можно: а) самостоятельно реализовать LSTM (интерфейса LSTMCell не хватит) б) снова воспользоваться трюком с установкой весов (но тут мы опираемся на свойство hanh(0)=0, к тому же, трюк в данном случае выглядит менее тривиально, чем с дропаутом Гала). Предлагается реализовать дрпоаут по сценираю а. Допишите класс:

In [31]:
class HandmadeLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.):
        super(HandmadeLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.input_weights = nn.Linear(input_size, 4 * hidden_size)
        self.hidden_weights = nn.Linear(hidden_size, 4 * hidden_size)
        
        self.reset_params()


    def reset_params(self):
        """
        initialization as in Pytorch
        do not forget to call this method!
        """
        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(0, stdv)
            

    def forward(self, inp, hidden=None):
        ### your code here
        # use functions init_h0_c0 and gen_dropout_masks defined above
        pass

Протестируйте вашу реализацию без дропаута (проконтролируйте качество и сравните время обучения с временем обучения nn.LSTM и RNNLayer), а также с dropout=0.5. Сравните качество модели с таким дропаутом с качеством модели с дропаутом Гала и Гарамани.

### Zoneout
Это еще одна модификация идеи дропаута применительно к рекуррентным нейросетям. В Zoneout на каждом временном шаге с вероятностью p компонента скрытого состояния обновляется, а с вероятностью 1-p берется с предыдущего шага. 
В Виде формул (m^t_h - бинарная маска):
 
(сначала обычный рекуррентный переход, например LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$
Затем Zoneout:
$$
h_t = h_t * m_h^t + h_{t-1}*(1-m_h^t)
$$
В этом методе маска уже должна быть разная во все моменты времени (иначе метод упрощается до дропаута Гала и Гарамани). На входы $x_t$ вновь можно накладывать маску до начала работы рекуррентного слоя.  

Если у вас осталось время, вы можете реализовать этот метод. Выберите основу из трех рассмотренных случаев самостоятельно.
